## Testing OpenAI's new Agents SDK

OpenAI announced on March 11th, 2025 new Responses API and Agents SDK (https://openai.com/index/new-tools-for-building-agents/) 

This notebook aims to test these new products

Main documentation: https://platform.openai.com/docs/guides/agents

In [2]:
import os

from dotenv import load_dotenv
load_dotenv()

import requests
from io import BytesIO
import base64

from pydantic import BaseModel, Field
import json

from IPython.display import display, Markdown
import time

from openai import OpenAI
client = OpenAI()

from agents import Agent, Runner, ModelSettings

#### Components

1. Agents

    - Very similar to Responses API - actually built on it.
    - Arguments (important ones):
        - `instructions` - developer/system message. Can also be dynamic with a function that takes as input the context and reads from it to fill a prompt template
        - `tools` - custom and built-in
        - `output_type` - structured output objects
        - `handoff_description` - description of the agent. This is used when the agent is used as a handoff, so that an LLM knows what it does and when to invoke it.
    - Run them via the `Runner` class (async by default)
        - handles e2e flow - no manual tool execution or handoff is needed. it is done once reached a `final_output` return

2. Tools
    - `function–tool` decorator - turn functions (well defined!) into JSON schemas

3. Handoffs
    - basically sub-agents (technically tools / functions) that are being called when an agent-to-agent handoff should happen
    - in handoffs, the new agent receives the **entire conversation history**, tools (or agents-as-tools) **only receive the generated output**

4. Output types ~ Structured outputs
    - pydantic Baseclasses passed to agent under `output_type`

5. Context
    - dataclass that is passed to all agents, handoffs, tools
    - everyone can read from it, fill prompt templates, pass parameters

6. Additionals
    1. Guardrails
    2. Lifecycle events (hooks)

#### First tests - very basic


In [6]:
async def main(agent, user_input):
    
    result = await Runner.run(agent, user_input)
    print(result.final_output)

    return result
    
agent = Agent(
    name="Funny guy", 
    instructions="You are a helpful assistant that always responds in a humorous manner",
    model = "gpt-4o",
    model_settings = ModelSettings(temperature=0.0),
    #handoff_description = None, # A description of the agent.
    #handoffs = [], # Sub-agents that the agent can delegate to. 
    #tools = [], # A list of tools that the agent can use.
    #input_guardrails = [],
    #output_guardrails = [],
    #output_type = None, # The type of the output object. If not provided, the output will be `str`.
    )

user_input = "Write a haiku about recursion in programming."

In [7]:
r = await main(agent, user_input)

Code calls itself, loops,  
In endless self-reference—  
Oops, stack overflow!


In [17]:
r.__dict__

{'input': 'Write a haiku about recursion in programming.',
 'new_items': [MessageOutputItem(agent=Agent(name='Funny guy', instructions='You are a helpful assistant that always responds in a humorous manner', handoff_description=None, handoffs=[], model='gpt-4o', model_settings=ModelSettings(temperature=0.0, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=False, truncation=None, max_tokens=None), tools=[], input_guardrails=[], output_guardrails=[], output_type=None, hooks=None), raw_item=ResponseOutputMessage(id='msg_67d57700ae888192a8858fe6a242451c084aa909d0d06bfd', content=[ResponseOutputText(annotations=[], text='Code calls itself, loops,  \nIn endless self-reference—  \nOops, stack overflow!', type='output_text')], role='assistant', status='completed', type='message'), type='message_output_item')],
 'raw_responses': [ModelResponse(output=[ResponseOutputMessage(id='msg_67d57700ae888192a8858fe6a242451c084aa909d0d06bfd', content=[Respons